# Procedures - Changing keys to their corresponding catalog descriptions 

## Import libraries

In [1]:
import pandas as pd
import numpy as np

## Load datasets from Procedures

In [2]:
file = '../../Data/Databases/Data Integration/procedimientos2010_2022.csv'
procedimientos = pd.read_csv(file, on_bad_lines='skip')
procedimientos.head()

,ID,NUMPROMED,TIPO,PROMED,ANEST,QUIROF,QH,QM
0,10952,1,Q,5411,2,1,99.0,99.0
1,23436,1,Q,5411,2,1,1.0,0.0
2,23659,1,Q,5411,2,1,1.0,0.0
3,23789,1,Q,5411,1,1,99.0,99.0
4,24367,1,Q,5411,2,1,1.0,0.0


In [3]:
procedimientos.shape

(519460, 8)

In [4]:
#Procedures keys
file = '../../Data/Keys/CLAVES_PROC.xlsx'
claves = pd.read_excel(file)
claves.head()

,CONSEC,CAPITULO,CATALOG_KEY,PRO_NOMBRE,PRO_CVE_EDIA,PRO_EDAD_IA,PRO_CVE_EDFA,PRO_EDAD_FA,SEX_TYPE,PROCEDIMIENTO_TYPE,PRO_PRINCIPAL,ACTUALIZACIONES_CIE-9MC,YEAR_MODIFI,YEAR_APLICACION,POR_NIVELA,PRO_SECCION,PRO_CATEGORIA,PRO_SUBCATEG,PRO_GRUPO_LC
0,1,0,00,"PROCEDIMIENTOS E INTERVENCIONES, NO CLASIFICAD...",0,0,0,0,0,NaN,NaN,NO,NO,NO,3,0,NaN,NaN,NaN
1,2,0,000,ULTRASONIDO TERAPÉUTICO,0,0,0,0,0,NaN,NO,NO,NO,NO,2,0,000,NaN,NaN
2,3,0,0001,ULTRASONIDO TERAPÉUTICO DE VASOS DE CABEZA Y C...,0,0,0,0,0,T,SI,NO,NO,NO,1,0,000,0001,63.1
3,4,0,0002,ULTRASONIDO TERAPÉUTICO CARDÍACO,0,0,0,0,0,T,SI,NO,NO,NO,1,0,000,0002,63.1
4,5,0,0003,ULTRASONIDO TERAPÉUTICO DE VASOS PERIFÉRICOS,0,0,0,0,0,T,SI,NO,NO,NO,1,0,000,0003,63.1


## Change keys from Medical Procedures

In [5]:
# Select the first 3 characters from the procedures 
procedimientos['PROMED'] = procedimientos['PROMED'].str[:3]

In [6]:
procedimientos.head()

,ID,NUMPROMED,TIPO,PROMED,ANEST,QUIROF,QH,QM
0,10952,1,Q,541,2,1,99.0,99.0
1,23436,1,Q,541,2,1,1.0,0.0
2,23659,1,Q,541,2,1,1.0,0.0
3,23789,1,Q,541,1,1,99.0,99.0
4,24367,1,Q,541,2,1,1.0,0.0


In [7]:
# Retain only the required columns
claves = claves[['CATALOG_KEY','PRO_NOMBRE']]
claves.rename(columns={'CATALOG_KEY':'CLAVE'}, inplace = True)
claves.head()

,CLAVE,PRO_NOMBRE
0,00,"PROCEDIMIENTOS E INTERVENCIONES, NO CLASIFICAD..."
1,000,ULTRASONIDO TERAPÉUTICO
2,0001,ULTRASONIDO TERAPÉUTICO DE VASOS DE CABEZA Y C...
3,0002,ULTRASONIDO TERAPÉUTICO CARDÍACO
4,0003,ULTRASONIDO TERAPÉUTICO DE VASOS PERIFÉRICOS


In [8]:
#Check duplicates
claves['CLAVE'].duplicated().sum()

16

In [9]:
#Drop duplicates
claves.drop_duplicates(subset = ['CLAVE'], keep = 'last', inplace = True)

In [10]:
# Merge keys table with procedures table
procedimientos.rename(columns={'PROMED':'CLAVE'}, inplace = True)
procedimientos = procedimientos.merge(claves, how = 'left', on = 'CLAVE')
procedimientos.rename(columns={'PRO_NOMBRE':'PROMED'}, inplace = True)
procedimientos.head()

,ID,NUMPROMED,TIPO,CLAVE,ANEST,QUIROF,QH,QM,PROMED
0,10952,1,Q,541,2,1,99.0,99.0,LAPAROTOMÍA
1,23436,1,Q,541,2,1,1.0,0.0,LAPAROTOMÍA
2,23659,1,Q,541,2,1,1.0,0.0,LAPAROTOMÍA
3,23789,1,Q,541,1,1,99.0,99.0,LAPAROTOMÍA
4,24367,1,Q,541,2,1,1.0,0.0,LAPAROTOMÍA


In [11]:
procedimientos.shape

(519460, 9)

In [12]:
# Change ANEST keys to their description
procedimientos['ANEST'] = procedimientos['ANEST'].replace({1:'GENERAL', 2:'REGIONAL', 3:'SEDACION', 4:'LOCAL', 
                                             5:'COMBINADA', 6:'NO USO', 9:'NE' })

In [13]:
# Change QUIROF keys to their description
procedimientos['QUIROF'] = procedimientos['QUIROF'].replace({1:'EN', 2:'FUERA', 9:'NE'})

In [14]:
# Change TIPO keys to their description
procedimientos['TIPO'] = procedimientos['TIPO'].replace({'Q':'QUIRURGICO', 'D':'DIAGNOSTICO', 
                                           'T':'TERAPEUTICO', 'F': 'MIXTO'})

In [15]:
# Drop key column
procedimientos = procedimientos[['ID','NUMPROMED','TIPO','PROMED','ANEST','QUIROF','QH','QM']]
procedimientos.head()

,ID,NUMPROMED,TIPO,PROMED,ANEST,QUIROF,QH,QM
0,10952,1,QUIRURGICO,LAPAROTOMÍA,REGIONAL,EN,99.0,99.0
1,23436,1,QUIRURGICO,LAPAROTOMÍA,REGIONAL,EN,1.0,0.0
2,23659,1,QUIRURGICO,LAPAROTOMÍA,REGIONAL,EN,1.0,0.0
3,23789,1,QUIRURGICO,LAPAROTOMÍA,GENERAL,EN,99.0,99.0
4,24367,1,QUIRURGICO,LAPAROTOMÍA,REGIONAL,EN,1.0,0.0


## Export to csv

In [16]:
from pathlib import Path 
filepath = Path('../../Data/Databases/Keys/procedimientos.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
procedimientos.to_csv(filepath, index = False)